In [4]:
import gzip


def unzip(input, output):

    input_file = input
    output_file = output

    with gzip.open(input_file, "rb") as f_in:
        with open(output_file, "wb") as f_out:

            f_out.write(f_in.read())

    print("file unzipped and save at", output_file)

unzip("/data/LSY/z_preparing_and_parts/commontrait_Colitis/index.html?acc=GSE87648&format=file&file=GSE87648_processedMethWb.txt.gz",
      "/data/LSY/z_preparing_and_parts/commontrait_Colitis/GSE87648_series_matrix.txt")

file unzipped and save at /data/LSY/z_preparing_and_parts/commontrait_Colitis/GSE87648_series_matrix.txt


In [5]:

import pandas as pd



# read matrix
with open(
    "/data/LSY/z_preparing_and_parts/commontrait_Colitis/GSE32148_series_matrix.txt", "r"
) as file:
    lines = file.readlines()

# find metadata
first_empty_line_index = lines.index("\n")
table_begin_index = next(
(
        i
        for i, line in enumerate(lines)
        if line.startswith("!series_matrix_table_begin")
    ),
    None,
)
end_index = table_begin_index if table_begin_index is not None else len(lines)
content_before_table = lines[first_empty_line_index:end_index]


# read metadata
metadata_lines = [
    line.strip()[1:].split("\t")
    for line in content_before_table
    if line.startswith("!")
]
metadata_df = pd.DataFrame(metadata_lines).T


metadata_df.columns = metadata_df.iloc[0]
metadata_df = metadata_df.drop(0, axis=0)

metadata_df.dropna(inplace=True)


In [41]:
# mask = metadata_df.iloc[:, 7] == '"Peripheral blood"'

# metadata_df = metadata_df[mask]



In [7]:
print(metadata_df.iloc[:,12].unique())

['"disease state: Crohn\'s disease"' '"disease state: normal"'
 '"disease state: ulcerative colitis"']


In [8]:

# find status
metadata_df["status"] = metadata_df.iloc[:, 12].apply(
    lambda x: 1 if "colitis" in x else 0 if "normal" in x else None
)
metadata_df = metadata_df.set_index("Sample_geo_accession")

In [9]:

metadata_df = metadata_df["status"].apply(pd.to_numeric, errors="coerce")

In [14]:
metadata_df

Sample_geo_accession
"GSM796660"    NaN
"GSM796661"    0.0
"GSM796662"    0.0
"GSM796663"    NaN
"GSM796664"    NaN
"GSM796665"    0.0
"GSM796666"    0.0
"GSM796667"    0.0
"GSM796668"    0.0
"GSM796669"    0.0
"GSM796670"    0.0
"GSM796671"    0.0
"GSM796672"    0.0
"GSM796673"    0.0
"GSM796674"    0.0
"GSM796675"    0.0
"GSM796676"    0.0
"GSM796677"    0.0
"GSM796678"    0.0
"GSM796679"    0.0
"GSM796680"    NaN
"GSM796681"    NaN
"GSM796682"    NaN
"GSM796683"    NaN
"GSM796684"    NaN
"GSM796685"    NaN
"GSM796686"    NaN
"GSM796687"    NaN
"GSM796688"    NaN
"GSM796689"    NaN
"GSM796690"    NaN
"GSM796691"    NaN
"GSM796692"    NaN
"GSM796693"    NaN
"GSM796694"    1.0
"GSM796695"    0.0
"GSM796696"    0.0
"GSM796697"    1.0
"GSM796698"    1.0
"GSM796699"    0.0
"GSM796700"    1.0
"GSM796701"    1.0
"GSM796702"    1.0
"GSM796703"    1.0
"GSM796704"    1.0
"GSM796705"    1.0
"GSM796706"    1.0
"GSM796707"    1.0
Name: status, dtype: float64

In [11]:


# read data
data_lines = [line.strip().split() for line in lines if not line.startswith("!")]
data_df = pd.DataFrame(data_lines).T

data_df.columns = data_df.iloc[0]
data_df = data_df.drop(0, axis=0)

data_df = data_df.set_index('"ID_REF"')
data_df = data_df.iloc[:, 1:]
data_df.columns = [col.strip('"') for col in data_df.columns]

data_df = data_df.apply(pd.to_numeric, errors="coerce")





In [12]:
print(data_df)

             cg00000029  cg00000108  cg00000109  cg00000165  cg00000236  \
"ID_REF"                                                                  
"GSM796660"     0.53599     0.91759     0.85866     0.19967     0.74830   
"GSM796661"     0.51288     0.90251     0.82595     0.19483     0.72959   
"GSM796662"     0.50556     0.91070     0.84799     0.19867     0.74411   
"GSM796663"     0.53280     0.92180     0.88189     0.17476     0.73424   
"GSM796664"     0.52627     0.91726     0.86505     0.21113     0.74834   
"GSM796665"     0.52920     0.92142     0.87332     0.20954     0.77799   
"GSM796666"     0.58514     0.90653     0.88042     0.17413     0.74483   
"GSM796667"     0.45767     0.92361     0.86524     0.17434     0.68453   
"GSM796668"     0.51203     0.92284     0.88565     0.20750     0.73487   
"GSM796669"     0.45172     0.91231     0.83913     0.19702     0.67801   
"GSM796670"     0.50633     0.91957     0.85846     0.16800     0.68883   
"GSM796671"     0.59065  

In [9]:

def m_to_beta(M):
    return 1 / (1 + 2 ** (-M))


In [10]:
# data_df = data_df.apply(m_to_beta)

In [15]:

# merge metadata and data
data_df["status"] = metadata_df
# drop other disease sample
data_df.dropna(subset=["status"], inplace=True)

In [24]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data_df, test_size=0.2, random_state=42)


# save data
train_data.to_csv(
    "/data/LSY/z_preparing_and_parts/commontrait_Multisclerosis/data_train.txt",
    index=True,
    header=True,
    sep="\t",
)


# save data
test_data.to_csv(
    "/data/LSY/z_preparing_and_parts/commontrait_Multisclerosis/data_test.txt",
    index=True,
    header=True,
    sep="\t",
)


In [16]:

# save data
data_df.to_csv(
    "/data/LSY/z_preparing_and_parts/commontrait_Colitis/data_valid.txt",
    index=True,
    header=True,
    sep="\t",
)
